In [1]:
import cv2 as cv
import os
import pandas as pd
import numpy as np
import json
import pickle
import time

In [90]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import LatentDirichletAllocation as LDA, PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report

In [3]:
import winsound
duration = 800  # milliseconds
freq = 300  # Hz
winsound.Beep(freq, duration)

In [95]:
#Load the model
#pickle.loads(kmeans_batch, open("KMeans_All_Trial_1.sav", 'wb'))
#Extracting HoG Features
hog_path = "\\".join((os.getcwd(), "HOG_features"))
hog_files = os.listdir(hog_path)
hog_aug_files = os.listdir("\\".join((os.getcwd(), "HOG_AUGMENTED")))

In [97]:
print(hog_files)
print()
print(hog_aug_files)

['cars_1_6554_hog_label_train.txt', 'cars_1_819_hog_label_val.txt', 'cars_1_820_hog_label_test.txt', 'nocars_1_6776_hog_label_train.txt', 'nocars_1_847_hog_label_test.txt', 'nocars_1_847_hog_label_val.txt']

['cars_1_783_hog_label_testmixed.txt', 'cars_1_790_hog_label_testbrightness.txt', 'cars_1_802_hog_label_testflipping.txt', 'cars_1_821_hog_label_testzooming.txt', 'cars_1_846_hog_label_testrotating.txt', 'nocars_1_812_hog_label_testmixed.txt', 'nocars_1_836_hog_label_testzooming.txt', 'nocars_1_843_hog_label_testrotating.txt', 'nocars_1_844_hog_label_testbrightness.txt', 'nocars_1_854_hog_label_testflipping.txt']


'i:\\WPI\\Fall 2020\\Computer Vision - RBE 549\\CV Project\\PCA_HOG\\HOG_AUGMENTED\\cars_1_790_hog_label_testbrightness.txt'

In [77]:
train_car_file = "\\".join((os.getcwd(), "HOG_features", hog_files[0]))
train_nocar_file = "\\".join((os.getcwd(), "HOG_features", hog_files[3]))
val_car_file = "\\".join((os.getcwd(), "HOG_features", hog_files[1]))
val_nocar_file = "\\".join((os.getcwd(), "HOG_features", hog_files[-1]))
val_car

In [78]:
train_car_fd = pd.DataFrame(pickle.load(open(train_car_file, 'rb')))
train_nocar_fd = pd.DataFrame(pickle.load(open(train_nocar_file, 'rb')))

val_car_fd = pd.DataFrame(pickle.load(open(val_car_file, 'rb')))
val_nocar_fd = pd.DataFrame(pickle.load(open(val_nocar_file, 'rb')))

In [112]:
val_carBright = pd.DataFrame(pickle.load(open(val_car_bright, 'rb')))
val_nocarBright = pd.DataFrame(pickle.load(open(val_nocar_bright, 'rb')))

In [31]:
#Each descriptor has 8192 features and there is one image per descriptor

8192

In [79]:
train_fd = pd.concat([train_car_fd, train_nocar_fd])
val_fd = pd.concat([val_car_fd, val_nocar_fd])

In [80]:
train_fd = shuffle(train_fd)
valid_fd = shuffle(val_fd)

In [98]:
n_component = 500
X_train = train_fd.iloc[:, 1]
Y_train = train_fd.iloc[:, 0].values
X_train = np.stack(X_train, axis = 0).astype(float)
X_val = valid_fd.iloc[:, 1]
Y_val = valid_fd.iloc[:, 0].values
X_val = np.stack(X_val, axis = 0).astype(float)


In [116]:
bright_fd = shuffle(pd.concat([val_carBright, val_nocarBright]))
X_bright = bright_fd.iloc[:, 1]
X_bright = np.stack(X_bright, axis = 0).astype(float)
Y_bright = bright_fd.iloc[:, 0].values

In [99]:
X_train[:2]

array([[0.54621013, 0.        , 0.3742874 , ..., 0.        , 0.        ,
        0.        ],
       [0.57023475, 0.12372745, 0.23160369, ..., 0.26805527, 0.01198031,
        0.        ]])

In [100]:
X_val[:2]

array([[0.6316689 , 0.        , 0.24929982, ..., 0.02359472, 0.01217506,
        0.29340658],
       [0.64673586, 0.26792491, 0.21433993, ..., 0.01877173, 0.04044378,
        0.        ]])

In [101]:
start_time = time.time()
pca_iter = PCA(n_components = n_component)
X_train_pca = pca_iter.fit_transform(X_train)
print("PCA: ".format(pca_iter))
# print("Last Component Variance: {:.4f}".format(pca_iter.explained_variance_[-1]))
print("Time: {:.4f}".format(time.time() - start_time))

PCA: 
Time: 27.9331


In [102]:
print("Last Component Variance: {:.4f}".format(pca_iter.explained_variance_[-1]))


Last Component Variance: 0.0750


In [103]:
pca_name = "_".join(("PCA_HoG","C"+str(n_component)+".sav"))
print(pca_name)
pickle.dump(pca_iter, open(pca_name, 'wb'))

PCA_HoG_C500.sav


In [104]:
start_time = time.time()
model_svm = SVC()
model_svm.fit(X_train_pca, Y_train)
print("Time {:.2f}".format(time.time() - start_time))

Time 46.39


In [105]:
X_val_pca = pca_iter.transform(X_val)

In [106]:
model_svm.score(X = X_val_pca, y = Y_val)

0.9591836734693877

In [107]:
Y_val_predict = model_svm.predict(X_val_pca)
Y_valid = Y_val

In [108]:
print("For Components: ", n_component)
print("Last Component Variance with components: {} = \t\t{:.4f}".format(n_component, pca_iter.explained_variance_[-1]))
print("")
print("F1_score for Validation with components: {} = \t\t{:.4f}".format(n_component, f1_score(y_true = Y_valid, y_pred = Y_val_predict)) )
print("Accuracy for Validation with components: {} = \t\t{:.4f}".format(n_component, accuracy_score(y_true = Y_valid, y_pred = Y_val_predict)) )
print("Precision for Validation with components: {} = \t{:.4f}".format(n_component, precision_score(y_true = Y_valid, y_pred = Y_val_predict)) )
print("Recall for Validation with components: {} = \t\t{:.4f}".format(n_component, recall_score(y_true = Y_valid, y_pred = Y_val_predict)) )
print("AUC Score for Validation with components: {} = \t{:.4f}".format(n_component, roc_auc_score(y_true = Y_valid, y_score = Y_val_predict)) )

For Components:  500
Last Component Variance with components: 500 = 		0.0750

F1_score for Validation with components: 500 = 		0.9585
Accuracy for Validation with components: 500 = 		0.9592
Precision for Validation with components: 500 = 	0.9585
Recall for Validation with components: 500 = 		0.9585
AUC Score for Validation with components: 500 = 	0.9592


In [122]:
hog_aug_files

['cars_1_783_hog_label_testmixed.txt',
 'cars_1_790_hog_label_testbrightness.txt',
 'cars_1_802_hog_label_testflipping.txt',
 'cars_1_821_hog_label_testzooming.txt',
 'cars_1_846_hog_label_testrotating.txt',
 'nocars_1_812_hog_label_testmixed.txt',
 'nocars_1_836_hog_label_testzooming.txt',
 'nocars_1_843_hog_label_testrotating.txt',
 'nocars_1_844_hog_label_testbrightness.txt',
 'nocars_1_854_hog_label_testflipping.txt']

In [144]:
cat_aug =  hog_aug_files[3]
car_aug = "\\".join((os.getcwd(), "HOG_AUGMENTED", hog_aug_files[4]))
nocar_aug = "\\".join((os.getcwd(), "HOG_AUGMENTED", hog_aug_files[-3]))

In [145]:
carAug = pd.DataFrame(pickle.load(open(car_aug, 'rb')))
nocarAug = pd.DataFrame(pickle.load(open(val_nocar_bright, 'rb')))

In [146]:
aug_fd = shuffle(pd.concat([carAug , nocarAug]))
X_aug = aug_fd.iloc[:, 1]
X_aug = np.stack(X_aug, axis = 0).astype(float)
Y_aug  = aug_fd.iloc[:, 0].values

In [147]:
X_aug_pca = pca_iter.transform(X_aug)
Y_aug_predict = model_svm.predict(X_aug_pca)

In [148]:
print("For Components for Aug: ", n_component)
print(cat_aug)
print("Last Component Variance with components: {} = \t\t{:.4f}".format(n_component, pca_iter.explained_variance_[-1]))
print("")
print("F1_score for Validation with components: {} = \t\t{:.4f}".format(n_component, f1_score(y_true = Y_aug, y_pred = Y_aug_predict)))
print("Accuracy for Validation with components: {} = \t\t{:.4f}".format(n_component, accuracy_score(y_true = Y_aug, y_pred = Y_aug_predict)))
print("Precision for Validation with components: {} = \t{:.4f}".format(n_component, precision_score(y_true = Y_aug, y_pred = Y_aug_predict)))
print("Recall for Validation with components: {} = \t\t{:.4f}".format(n_component, recall_score(y_true = Y_aug, y_pred = Y_aug_predict)))
print("AUC Score for Validation with components: {} = \t{:.4f}".format(n_component, roc_auc_score(y_true = Y_aug, y_score = Y_aug_predict)))

For Components for Aug:  500
cars_1_821_hog_label_testzooming.txt
Last Component Variance with components: 500 = 		0.0750

F1_score for Validation with components: 500 = 		0.8624
Accuracy for Validation with components: 500 = 		0.8757
Precision for Validation with components: 500 = 	0.9676
Recall for Validation with components: 500 = 		0.7778
AUC Score for Validation with components: 500 = 	0.8759
